# Import Section
---

In [1]:
import shutil
import os
from pathlib import Path
from subprocess import Popen

import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
from ipywidgets import AppLayout, Button, Layout, Box, FloatText, Textarea, Dropdown, Label, IntSlider
from IPython.display import display, HTML
from IPython.display import clear_output
from ipyfilechooser import FileChooser

# Widgets Control Section
---

In [4]:
class train_config_and_cmds_widgets():
    def __init__(self):

        self.cfg_dir = "cfg"
        self.my_cfg_path = Path(self.cfg_dir) / "my_config.yaml"
        self.m_src_tflite = None
        
        self.tflite_file_loc = None
        self.src_audio_test_file = None
        
        form_item_layout = Layout(
        display='flex',
        flex_flow='row',
        justify_content='space-between',
        )
        
        ### train ###
        self.A_ta = widgets.Text(value='yolox_n_nu_320', placeholder='Type something', disabled=False)
        self.E_ta = widgets.BoundedIntText(value=320, min=96, max=416, step=1, disabled=False)
        self.B_ta = widgets.IntSlider(value=64, min=4, max=128, step=4)
        self.C_ta = widgets.BoundedIntText(value=100, min=5, max=10000, step=1, disabled=False)
        self.D_ta = widgets.BoundedIntText(value=6, min=1, max=1000, step=1, disabled=False)
        self.J_ta = Textarea(value='datasets/medicinev2_coco', disabled=False)
        self.K_ta = Textarea(value='medicinev2_train.json', disabled=False)
        self.L_ta = Textarea(value='medicinev2_val.json', disabled=False)
        self.F_ta = widgets.Checkbox(value=True, disabled=False, indent=False)
        self.G_ta = Textarea(value='pretrain/tflite_yolox_nano_ti/320_DW/yolox_nano_320_DW_ti_lite.pth', disabled=False)
        
        form_train_items = [
            Box([Label(value = 'Project Name'),                          self.A_ta], layout=form_item_layout),
            Box([Label(value = 'Image Size (WxH)'),                      self.E_ta], layout=form_item_layout),
            Box([Label(value = 'Batch Size'),                            self.B_ta], layout=form_item_layout),
            Box([Label(value = 'Epochs'),                                self.C_ta], layout=form_item_layout),
            Box([Label(value = 'Number of Classes'),                     self.D_ta], layout=form_item_layout),
            Box([Label(value = 'Dataset Dir'),                           self.J_ta], layout=form_item_layout),
            Box([Label(value = 'Train Annotation File'),                 self.K_ta], layout=form_item_layout),
            Box([Label(value = 'Validation Annotation File'),            self.L_ta], layout=form_item_layout),
            Box([Label(value = 'Use Pretrain Model'),                    self.F_ta], layout=form_item_layout),
            Box([Label(value = 'Pretrain Model File'),                    self.G_ta], layout=form_item_layout),
        ]
        
        self.form_output_train_cmd = Box(form_train_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 1px lightblue',
            align_items='stretch',
            width='100%',
        ))

        ### run buttoms box ###
        self.A_rb = widgets.Checkbox(value=True, disabled=False, indent=False)
        self.B_rb = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')

        form_run_buttoms_items = [
            Box([Label(value = 'Directly Run on Jupyter'), self.A_rb], layout=form_item_layout),
            Box([Label(value = 'Start to Train'), self.B_rb], layout=form_item_layout),
        ]

        self.form_run_buttoms_cmd = Box(form_run_buttoms_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 1px lightblue',
            align_items='stretch',
            width='100%',
        ))

        ### test ###
        self.A_tt = widgets.Button(description='Setting', layout=Layout(width='30%', height='30px'), button_style='success')
        self.C_tt = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')
        self.B_tt = Textarea(value='datasets/medicinev2_coco', disabled=False)
        self.D_tt = Textarea(value='medicinev2_val.json', disabled=False)
      
        form_test_items = [
            Box([Label(value = 'Choose the tflite file'), self.A_tt], layout=form_item_layout),
            Box([Label(value = 'Dataset Dir'),            self.B_tt], layout=form_item_layout),
            Box([Label(value = 'Test Annotation File'),   self.D_tt], layout=form_item_layout),
            Box([Label(value = 'Start to Test'),          self.C_tt], layout=form_item_layout),
        ]
        
        self.form_output_test_cmd = Box(form_test_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='50%',
        ))
        
        ### convert model cpp ###
        self.A_cm = widgets.Text(value=r'YOLOX_outputs/yolox_n_nu_320', placeholder='Type something', disabled=False)
        self.B_cm = widgets.Text(value=r'datasets/coco/train2017', placeholder='Type something', disabled=False)
        self.C_cm = widgets.Text(value='yolox_n_nu_320_mytask', placeholder='Type something', disabled=False)
        self.D_cm = widgets.Button(description='Gen Bat file', layout=Layout(width='30%', height='30px'), button_style='success')
      
        form_convert_items_paths =  [
            Box([Label(value = 'Project Dir'),          self.A_cm], layout=form_item_layout),
            Box([Label(value = 'Train Dataset Dir'),         self.B_cm], layout=form_item_layout),
            Box([Label(value = 'Your Modle File Name'),            self.C_cm], layout=form_item_layout)
        ]
        
        form_convert_items = [
            Box(form_convert_items_paths, layout=Layout(
            display='flex',
            flex_flow='column',
            justify_content ='center',    
            border='dotted 3px lightblue',
            align_items='stretch',
            width='70%')),
            Box([Label(value = 'Convert to tflite int8 & Vela'), self.D_cm], layout=form_item_layout),
        ]
        
        self.form_output_convert_cmd = Box(form_convert_items, layout=Layout(
            display='flex',
            flex_flow='column',
            justify_content ='center',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='70%',
        ))
        
    def move_allfiles(self, src_folder, dst_folder):
        copy_num = 0
        
        files = os.listdir(src_folder)
        for f in files:
            fullpath = os.path.join(src_folder, f)
            if os.path.isdir(fullpath):  #copy whole folder
                shutil.move(fullpath, dst_folder)
                print("Copy finish: {}".format(f))
    
    def show_headline(self, output):
        html0= widgets.HTML(value = f"<b><font color='lightblue'><font size=4>{output}</b>")
        display(html0)
    
    def show_main(self):   
        
        intro_text = 'Yolox-nano-nu Easy Setting'
        htmlWidget = widgets.HTML(value = f"<b><font color='lightgreen'><font size=6>{intro_text}</b>")
        display(htmlWidget)
        
        #Create an accordion and put the 2 boxes
        accordion = widgets.Accordion(children=[self.form_output_train_cmd
                                                ]).add_class("parentstyle")
        display(HTML("<style>.parentstyle > .p-Accordion-child > .p-Collapse-header{background-color:green}</style>"))
        accordion.set_title(0, 'Configure the Training')
        
        # Create a box combining with 2 elements
        box_data_train = Box([accordion, self.form_run_buttoms_cmd], layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='50%',
        ))
        
        #Create a tab and put the 2 boxes
        tab = widgets.Tab(children=[box_data_train, self.form_output_test_cmd, self.form_output_convert_cmd]).add_class("parentstyle")
        tab_contents = ['Train', 'Test', 'Deployment']
        tab.titles = tab_contents
       
        
        #Add a custom style tag to the notebook, you can use dev tool to inspect the class names
        #display(HTML("<style>.parentstyle > .p-Accordion-child > .p-Collapse-header{background-color:green}</style>"))
        #accordion.set_title(0, 'Configure the Training')        
        output_widgets = widgets.Output(layout=Layout(border = '1px solid green'))

        def act_para(
                     A_ta, B_ta, C_ta, D_ta, E_ta, J_ta, K_ta, L_ta,
                     A_cm, B_cm, C_cm, A_rb
                     ):
        
            # If any value is changed, clear the widgets
            with output_widgets:
                output_widgets.clear_output()
                
            #if data_exist:
            #    self.form_data_prepare_cmd.layout.visibility = 'hidden'
            #else:
            #    self.form_data_prepare_cmd.layout.visibility = 'visible'
        
        #------------------#
        # widgets.Accordion's interactive input with action function `act_para()`
        #------------------#
        out_inter = widgets.interactive_output(act_para, {
                                                          'A_ta': self.A_ta, 'B_ta': self.B_ta, 'C_ta': self.C_ta,
                                                          'D_ta': self.D_ta, 'E_ta': self.E_ta, 
                                                          'J_ta': self.J_ta, 'K_ta': self.K_ta, 'L_ta': self.L_ta,
                                                          'A_cm': self.A_cm, 'B_cm': self.B_cm, 'C_cm': self.C_cm,
                                                          'A_rb': self.A_rb
                                                          })

        display(tab, out_inter)
        
        #------------------#
        # for labelimg cmd, move to outside of act_para to prevent keep trigering
        #------------------#
        #output_widgets = widgets.Output(layout=Layout(border = '1px solid green'))
        display(output_widgets)
        
        def on_button_clicked_train(b):
                with output_widgets:
                    clear_output()
                    #print("Train. . .")
                    self.run_train()       
        self.B_rb.on_click(on_button_clicked_train)
        
        def on_button_clicked_cpp(b):
                with output_widgets:
                    clear_output()
                    self.convert_tflu()       
        self.D_cm.on_click(on_button_clicked_cpp)
        
        def on_button_clicked_choose_tflite_test(b):
                with output_widgets:
                    clear_output()
                    self.choose_tflite()       
        self.A_tt.on_click(on_button_clicked_choose_tflite_test)
        
        def on_button_clicked_test(b):
            with output_widgets:
                clear_output()
                self.run_test_tflite()           
        self.C_tt.on_click(on_button_clicked_test)

    def choose_tflite(self):
        path_ftflite = os.path.join(os.getcwd(), "YOLOX_outputs")
        f_tflite = FileChooser(path_ftflite)
        # Restrict navigation to /Users
        f_tflite.sandbox_path = os.getcwd()
        f_tflite.filter_pattern = ['*.tflite']
        f_tflite.title = f"<b><font color='lightblue'><font size=4>Choose the Tflite for test single audio.</b>"
        display(f_tflite)
        
        def act_test():
            work_dir_name = os.getcwd().split("\\")[-1]
            m_src_dir = f_tflite.selected_path.split(work_dir_name)[-1]
            m_src_tflite  = f_tflite.selected.split("\\")[-1]
            print("The chosen dir: {}".format(m_src_dir))
            print("The chosen tflite: {}".format(m_src_tflite))
            self.m_src_tflite = os.path.join(m_src_dir, m_src_tflite)
            print("Finish!")
        evt = interact_manual(act_test)
        evt.widget.children[0].description = 'Choose this tflite file'  #because there are 3 parameter of the evt
        evt.widget.children[0].button_style = 'primary'

    def run_train(self):

        if self.A_rb.value:
            if self.F_ta.value:
                %run tools/train.py -f exps/default/yolox_nano_ti_lite_nu.py -d 1 -b $self.B_ta.value --fp16 -o -c $self.G_ta.value -ename $self.A_ta.value -im $self.E_ta.value -epo $self.C_ta.value -nc $self.D_ta.value --data_dir $self.J_ta.value --train_ann $self.K_ta.value --val_ann $self.L_ta.value 
            else:
                %run tools/train.py -f exps/default/yolox_nano_ti_lite_nu.py -d 1 -b $self.B_ta.value --fp16 -o -ename $self.A_ta.value -im $self.E_ta.value -epo $self.C_ta.value -nc $self.D_ta.value --data_dir $self.J_ta.value --train_ann $self.K_ta.value --val_ann $self.L_ta.value 
        else:
            # get the training cmd
            self.show_headline('Please copy these commands to the CMD.exe Prompt to execute. ')
            
            self.show_headline('(1.) Please excute the below commands under the working directory')
            print("cd {}".format(os.path.join(os.getcwd())))    
            self.show_headline('(2.) Train:')
            if self.F_ta.value:
                train_cmd = r"python tools/train.py -f exps/default/yolox_nano_ti_lite_nu.py -d 1 -b {} --fp16 -o -c {} -ename {} -im {} -epo {} -nc {} --data_dir {} --train_ann {} --val_ann {}".format(self.B_ta.value, self.G_ta.value, self.A_ta.value, self.E_ta.value, self.C_ta.value, self.D_ta.value, self.J_ta.value, self.K_ta.value, self.L_ta.value)
            else:
                train_cmd = r"python tools/train.py -f exps/default/yolox_nano_ti_lite_nu.py -d 1 -b {} --fp16 -o -ename {} -im {} -epo {} -nc {} --data_dir {} --train_ann {} --val_ann {}".format(self.B_ta.value, self.A_ta.value, self.E_ta.value, self.C_ta.value, self.D_ta.value, self.J_ta.value, self.K_ta.value, self.L_ta.value)
            print(train_cmd)
            
    def run_test_tflite(self):
        
        print("The tflite file: {}".format(self.m_src_tflite))
        
        %run demo/TFLite/tflite_eval.py -m $self.m_src_tflite -v $self.B_tt.value -a $self.D_tt.value
        
        print("Finish !!")

    def update_line_in_file(self, file_path, search_phrase, new_value):
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        with open(file_path, 'w') as file:
            for line in lines:
                if line.startswith(search_phrase[0]):
                    file.write(f"{new_value[0]}\n")
                elif line.startswith(search_phrase[1]):
                    file.write(f"{new_value[1]}\n")
                elif line.startswith(search_phrase[2]):
                    file.write(f"{new_value[2]}\n")
                elif line.startswith(search_phrase[3]):
                    file.write(f"{new_value[3]}\n")
                elif line.startswith(search_phrase[4]):
                    file.write(f"{new_value[4]}\n")
                elif line.startswith(search_phrase[5]):
                    file.write(f"{new_value[5]}\n")    
                elif line.startswith(search_phrase[6]):
                    file.write(f"{new_value[6]}\n")    
                else:
                    file.write(line)
        
    def convert_tflu(self):
        
        print("Update the variables of yolox_convert.bat done!")
        
        file_path = "yolox_convert.bat"
        search_phrase = ["set IMG_SIZE", "set NUM_CLS", "set MODEL_FILE_NAME", "set OUTPUT_DIR", "set TRAIN_DATASET", "set IMAGE_SIZE_EXPR", "set CLASS_EXPR"]
        IMAGE_SIZE_update = f"set IMAGE_SIZE_EXPR=\"extern const int originalImageSize = {self.E_ta.value}\""
        CLASS_EXPR_update = f"set CLASS_EXPR=\"extern const int numClasses = {self.D_ta.value}\""
        new_value = [f"set IMG_SIZE={self.E_ta.value}", f"set NUM_CLS={self.D_ta.value}", f"set MODEL_FILE_NAME={self.C_cm.value}", f"set OUTPUT_DIR={self.A_cm.value}", f"set TRAIN_DATASET={self.B_cm.value}", IMAGE_SIZE_update, CLASS_EXPR_update]
        self.update_line_in_file(file_path, search_phrase, new_value)
        
        self.show_headline('Please execute yolox_convert.bat in current work directory.')
        
        
            

# Run Section
---

In [6]:
act = train_config_and_cmds_widgets()
act.show_main()

HTML(value="<b><font color='lightgreen'><font size=6>Yolox-nano-nu Easy Setting</b>")

Output()

Output(layout=Layout(border_bottom='1px solid green', border_left='1px solid green', border_right='1px solid g…